In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import time

import Featurizer as ft

warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')
%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
pd.set_option('display.max_columns', 50)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12.0,8.0]

In [4]:
import category_encoders as ce

OHE = ce.OneHotEncoder( handle_unknown='ignore', use_cat_names=True)

BE = ce.BinaryEncoder( handle_unknown='ignore')
SumE = ce.SumEncoder( handle_unknown='ignore')

TE1 = ce.TargetEncoder( handle_unknown='ignore')
TE2 = ce.TargetEncoder( handle_unknown='ignore')
TE3 = ce.TargetEncoder( handle_unknown='ignore')
TE4 = ce.TargetEncoder( handle_unknown='ignore')
TE5 = ce.TargetEncoder( handle_unknown='ignore')
TE6 = ce.TargetEncoder( handle_unknown='ignore')

In [5]:
df_train = pd.read_csv("../../data/palabras.csv")

#mx = pd.read_csv("../data/mx_modified.csv")

#mx = mx[["population_proper","provincia"]]

#df_train = pd.merge(df_train,mx,on="provincia",how="left")

#df_feature_descripcion = pd.read_csv("../data/word-features-reduced.csv", dtype=np.float16)

#df_train = df_train.join(df_feature_descripcion)

## Pre-processing

In [6]:
X = df_train.drop("precio", axis=1)
y = df_train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
X_train_submit = X_train.copy()

In [8]:
X_train["ambientes"] = X_train["banos"] + X_train["habitaciones"]
X_train["ambientesygarage"] = X_train["banos"] + X_train["habitaciones"]+X_train["garages"]

X_test["ambientes"] = X_test["banos"] + X_test["habitaciones"]
X_test["ambientesygarage"] = X_test["banos"] + X_test["habitaciones"]+X_test["garages"]

In [9]:
X_train[ ft.getTarget1Cols() ] = X_train[ft.getAllCols()+["antiguedad","banos","habitaciones"]]
X_test[ ft.getTarget1Cols() ] = X_test[ft.getAllCols()+["antiguedad","banos","habitaciones"]]

X_train[ ft.getTarget2Cols() ] = X_train[["tipodepropiedad","idzona"]]
X_test[ ft.getTarget2Cols() ] = X_test[["tipodepropiedad","idzona"]]

X_train[ ft.getTarget3Cols() ] = X_train[ft.getAllCols()]
X_test[ ft.getTarget3Cols() ] = X_test[ft.getAllCols()]

X_train[ ft.getTarget4Cols() ] = X_train[ft.getAllCols()]
X_test[ ft.getTarget4Cols() ] = X_test[ft.getAllCols()]

X_train[ ft.getTarget5Cols() ] = X_train[ft.getAllCols()]
X_test[ ft.getTarget5Cols() ] = X_test[ft.getAllCols()]

X_train[ ft.getTarget6Cols() ] = X_train[ft.getAllCols()]
X_test[ ft.getTarget6Cols() ] = X_test[ft.getAllCols()]

In [10]:
X_train = ft.preprocess(X_train, OHE, BE, 'train', TE1, TE2, TE3, TE4, SumE, TE5, TE6, y_train)
X_test = ft.preprocess( X_test , OHE, BE, 'test' , TE1, TE2, TE3, TE4, SumE, TE5, TE6)

--- 15.988898277282715 seconds ---
--- 2.698723554611206 seconds ---


In [11]:
print(f"Original shapes: X={X.shape} y={y.shape}")
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y=-{y_test.shape}")

Original shapes: X=(240000, 84) y=(240000,)
Train shapes: X=(180000, 217) y=(180000,)
Test  shapes: X=(60000, 217)  y=-(60000,)


In [12]:
X_train

,intercept,id,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa,tipodepropiedad_Apartamento,tipodepropiedad_Terreno,tipodepropiedad_Local Comercial,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Bodega comercial,tipodepropiedad_Otros,tipodepropiedad_Oficina comercial,tipodepropiedad_Terreno comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Edificio,tipodepropiedad_Duplex,tipodepropiedad_Terreno industrial,tipodepropiedad_Villa,tipodepropiedad_Nave industrial,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Departamento Compartido,tipodepropiedad_Rancho,tipodepropiedad_Huerta,tipodepropiedad_nan,tipodepropiedad_Lote,...,ciudad_target_banos,idzona_target_banos,tipodepropiedad_target_banos,provincia_target_banos,ciudad_target_habit,idzona_target_habit,tipodepropiedad_target_habit,provincia_target_habit,ciudad_target_banoshabit,idzona_target_banoshabit,tipodepropiedad_target_banoshabit,provincia_target_banoshabit,metroscubiertostotales,m_cuadrado,m_tot_log,m_cub_log,m2_log,m_cuad_log,m_tot_sqrt,m_cub_sqrt,m2_sqrt,banosPorHabit,mtotPorHabit,mtotPorBanos,mtotPorAmbiente
122812,1,298880,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.375197,2.286065,2.366924,2.326781,2.877769,2.914834,2.957374,2.841563,0.855810,0.807407,0.807171,0.856227,330.0,27200.0,5.075174,5.135798,5.799093,10.210972,12.649111,13.038405,18.165902,0.666667,53.333333,80.0,32.000000
180398,1,87355,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.090909,2.133796,2.242734,1.884764,3.069767,2.903655,3.093696,2.897596,0.727649,0.758639,0.743306,0.663843,296.0,21840.0,5.049856,4.941642,5.690359,9.991498,12.489996,11.832160,17.204651,1.000000,78.000000,78.0,39.000000
157317,1,141560,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.919353,1.763636,1.817604,1.985042,2.536850,2.378571,2.388201,2.741933,0.794755,0.789767,0.777893,0.760274,186.0,8649.0,4.532599,4.532599,5.225747,9.065199,9.643651,9.643651,13.638182,0.666667,31.000000,46.5,18.600000
78425,1,63104,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.609076,1.217391,2.242734,2.083613,2.818333,2.727273,3.093696,2.972207,0.582041,0.469697,0.743306,0.717504,198.0,9605.0,4.442651,4.727388,5.288267,9.170039,9.219544,10.630146,14.071247,0.333333,28.333333,85.0,21.250000
73755,1,260795,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.081290,2.318182,2.242734,1.985042,3.037325,3.042254,3.093696,2.741933,0.708725,0.771717,0.743306,0.760274,550.0,60000.0,5.991465,5.010635,6.309918,11.002100,20.000000,12.247449,23.452079,0.500000,100.000000,200.0,66.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1,271551,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.375197,3.148148,2.242734,2.326781,2.877769,3.090909,3.093696,2.841563,0.855810,1.086420,0.743306,0.856227,407.0,40950.0,5.416100,5.204007,6.008813,10.620107,15.000000,13.490738,20.174241,1.000000,75.000000,75.0,37.500000
103694,1,151225,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.356115,1.000000,2.242734,2.083613,2.444444,2.052632,3.093696,2.972207,0.569349,0.491228,0.743306,0.717504,118.0,3481.0,4.077537,4.077537,4.770685,8.155075,7.681146,7.681146,10.862780,0.500000,29.500000,59.0,19.666667
131932,1,245245,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.005226,2.062500,2.242734,2.229520,2.840351,2.562500,3.093696,3.039390,0.735663,0.830645,0.743306,0.752466,210.0,11025.0,4.653960,4.653960,5.347108,9.307921,10.246951,10.246951,14.491377,0.666667,35.000000,52.5,21.000000
146867,1,154866,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.356115,1.192053,2.242734,2.083613,2.444444,2.340000,3.093696,2.972207,0.569349,0.531444,0.743306,0.717504,160.0,6400.0,4.382027,4.382027,5.075174,8.764053,8.944272,8.944272,12.649111,0.500000,20.000000,40.0,13.333333


In [13]:
features = X_train.columns
X_train.columns = list(range(0,X_train.shape[1]))
X_test.columns = list(range(0,X_test.shape[1]))

## Training

In [16]:
import lightgbm as lgb
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

r1 = HistGradientBoostingRegressor(max_iter=1000,verbose=10,max_depth=10,
                                      max_leaf_nodes=60,random_state=42,
                                     )

r2 = lgb.LGBMRegressor(learning_rate=0.05,n_estimators=1000,
                          max_depth=14,subsample=0.5,n_jobs=3,verbose=10)

r3 = XGBRegressor(n_estimators=1500,n_jobs=6,objective="reg:squarederror",
                     max_depth= 11,min_child_weight = 1,colsample_bytree=0.5,
                     gamma=1, learning_rate=0.04, subsample=1)

model = VotingRegressor([('hist', r1), ('lgbm', r2),('xgb', r3)])

model

VotingRegressor(estimators=[('hist',
                             HistGradientBoostingRegressor(l2_regularization=0.0,
                                                           learning_rate=0.1,
                                                           loss='least_squares',
                                                           max_bins=256,
                                                           max_depth=10,
                                                           max_iter=1000,
                                                           max_leaf_nodes=60,
                                                           min_samples_leaf=20,
                                                           n_iter_no_change=None,
                                                           random_state=42,
                                                           scoring=None,
                                                           tol=1e-07,
                                           

In [17]:
start_time = time.time()
model.fit(X_train, y_train)

print("--- %s seconds ---" % (time.time() - start_time))

Binning 0.312 GB of data: 1.330 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 60 leaves, max depth = 9, in 0.233s
[2/1000] 1 tree, 60 leaves, max depth = 8, in 0.232s
[3/1000] 1 tree, 60 leaves, max depth = 9, in 0.226s
[4/1000] 1 tree, 60 leaves, max depth = 9, in 0.224s
[5/1000] 1 tree, 60 leaves, max depth = 8, in 0.231s
[6/1000] 1 tree, 60 leaves, max depth = 10, in 0.226s
[7/1000] 1 tree, 60 leaves, max depth = 9, in 0.225s
[8/1000] 1 tree, 60 leaves, max depth = 9, in 0.225s
[9/1000] 1 tree, 60 leaves, max depth = 8, in 0.233s
[10/1000] 1 tree, 60 leaves, max depth = 8, in 0.243s
[11/1000] 1 tree, 60 leaves, max depth = 10, in 0.238s
[12/1000] 1 tree, 60 leaves, max depth = 10, in 0.240s
[13/1000] 1 tree, 60 leaves, max depth = 9, in 0.245s
[14/1000] 1 tree, 60 leaves, max depth = 10, in 0.240s
[15/1000] 1 tree, 60 leaves, max depth = 10, in 0.210s
[16/1000] 1 tree, 60 leaves, max depth = 10, in 0.225s
[17/1000] 1 tree, 60 leaves, max depth = 9, in 0.220s
[18/1000] 1 tree, 

[148/1000] 1 tree, 60 leaves, max depth = 10, in 0.121s
[149/1000] 1 tree, 60 leaves, max depth = 10, in 0.143s
[150/1000] 1 tree, 60 leaves, max depth = 10, in 0.096s
[151/1000] 1 tree, 60 leaves, max depth = 10, in 0.119s
[152/1000] 1 tree, 60 leaves, max depth = 10, in 0.112s
[153/1000] 1 tree, 60 leaves, max depth = 10, in 0.127s
[154/1000] 1 tree, 60 leaves, max depth = 10, in 0.096s
[155/1000] 1 tree, 60 leaves, max depth = 10, in 0.114s
[156/1000] 1 tree, 60 leaves, max depth = 10, in 0.094s
[157/1000] 1 tree, 60 leaves, max depth = 10, in 0.131s
[158/1000] 1 tree, 477 leaves, max depth = 10, in 0.430s
[159/1000] 1 tree, 60 leaves, max depth = 10, in 0.107s
[160/1000] 1 tree, 60 leaves, max depth = 10, in 0.089s
[161/1000] 1 tree, 280 leaves, max depth = 10, in 0.266s
[162/1000] 1 tree, 60 leaves, max depth = 10, in 0.109s
[163/1000] 1 tree, 60 leaves, max depth = 10, in 0.104s
[164/1000] 1 tree, 60 leaves, max depth = 10, in 0.103s
[165/1000] 1 tree, 60 leaves, max depth = 10, 

[294/1000] 1 tree, 60 leaves, max depth = 10, in 0.108s
[295/1000] 1 tree, 60 leaves, max depth = 10, in 0.103s
[296/1000] 1 tree, 222 leaves, max depth = 10, in 0.230s
[297/1000] 1 tree, 60 leaves, max depth = 10, in 0.105s
[298/1000] 1 tree, 60 leaves, max depth = 10, in 0.103s
[299/1000] 1 tree, 60 leaves, max depth = 10, in 0.105s
[300/1000] 1 tree, 60 leaves, max depth = 10, in 0.091s
[301/1000] 1 tree, 60 leaves, max depth = 10, in 0.080s
[302/1000] 1 tree, 160 leaves, max depth = 10, in 0.186s
[303/1000] 1 tree, 60 leaves, max depth = 10, in 0.085s
[304/1000] 1 tree, 330 leaves, max depth = 10, in 0.329s
[305/1000] 1 tree, 60 leaves, max depth = 10, in 0.102s
[306/1000] 1 tree, 60 leaves, max depth = 10, in 0.089s
[307/1000] 1 tree, 60 leaves, max depth = 10, in 0.087s
[308/1000] 1 tree, 60 leaves, max depth = 10, in 0.123s
[309/1000] 1 tree, 235 leaves, max depth = 10, in 0.233s
[310/1000] 1 tree, 60 leaves, max depth = 10, in 0.083s
[311/1000] 1 tree, 60 leaves, max depth = 10

[440/1000] 1 tree, 60 leaves, max depth = 10, in 0.079s
[441/1000] 1 tree, 60 leaves, max depth = 10, in 0.111s
[442/1000] 1 tree, 328 leaves, max depth = 10, in 0.261s
[443/1000] 1 tree, 60 leaves, max depth = 10, in 0.087s
[444/1000] 1 tree, 60 leaves, max depth = 10, in 0.096s
[445/1000] 1 tree, 60 leaves, max depth = 10, in 0.076s
[446/1000] 1 tree, 60 leaves, max depth = 10, in 0.092s
[447/1000] 1 tree, 60 leaves, max depth = 10, in 0.087s
[448/1000] 1 tree, 269 leaves, max depth = 10, in 0.264s
[449/1000] 1 tree, 60 leaves, max depth = 10, in 0.106s
[450/1000] 1 tree, 60 leaves, max depth = 10, in 0.081s
[451/1000] 1 tree, 322 leaves, max depth = 10, in 0.291s
[452/1000] 1 tree, 150 leaves, max depth = 10, in 0.139s
[453/1000] 1 tree, 60 leaves, max depth = 10, in 0.107s
[454/1000] 1 tree, 60 leaves, max depth = 10, in 0.098s
[455/1000] 1 tree, 60 leaves, max depth = 10, in 0.115s
[456/1000] 1 tree, 60 leaves, max depth = 10, in 0.072s
[457/1000] 1 tree, 60 leaves, max depth = 10

[586/1000] 1 tree, 208 leaves, max depth = 10, in 0.194s
[587/1000] 1 tree, 60 leaves, max depth = 10, in 0.105s
[588/1000] 1 tree, 265 leaves, max depth = 10, in 0.267s
[589/1000] 1 tree, 60 leaves, max depth = 10, in 0.093s
[590/1000] 1 tree, 60 leaves, max depth = 10, in 0.118s
[591/1000] 1 tree, 60 leaves, max depth = 10, in 0.083s
[592/1000] 1 tree, 60 leaves, max depth = 10, in 0.109s
[593/1000] 1 tree, 259 leaves, max depth = 10, in 0.242s
[594/1000] 1 tree, 196 leaves, max depth = 10, in 0.216s
[595/1000] 1 tree, 324 leaves, max depth = 10, in 0.267s
[596/1000] 1 tree, 356 leaves, max depth = 10, in 0.317s
[597/1000] 1 tree, 147 leaves, max depth = 10, in 0.150s
[598/1000] 1 tree, 60 leaves, max depth = 10, in 0.089s
[599/1000] 1 tree, 60 leaves, max depth = 10, in 0.099s
[600/1000] 1 tree, 60 leaves, max depth = 10, in 0.080s
[601/1000] 1 tree, 60 leaves, max depth = 10, in 0.120s
[602/1000] 1 tree, 60 leaves, max depth = 10, in 0.111s
[603/1000] 1 tree, 237 leaves, max depth 

[732/1000] 1 tree, 285 leaves, max depth = 10, in 0.231s
[733/1000] 1 tree, 60 leaves, max depth = 10, in 0.101s
[734/1000] 1 tree, 60 leaves, max depth = 10, in 0.111s
[735/1000] 1 tree, 60 leaves, max depth = 10, in 0.088s
[736/1000] 1 tree, 60 leaves, max depth = 10, in 0.113s
[737/1000] 1 tree, 60 leaves, max depth = 10, in 0.084s
[738/1000] 1 tree, 60 leaves, max depth = 10, in 0.102s
[739/1000] 1 tree, 60 leaves, max depth = 10, in 0.107s
[740/1000] 1 tree, 233 leaves, max depth = 10, in 0.308s
[741/1000] 1 tree, 60 leaves, max depth = 10, in 0.080s
[742/1000] 1 tree, 60 leaves, max depth = 10, in 0.094s
[743/1000] 1 tree, 60 leaves, max depth = 10, in 0.099s
[744/1000] 1 tree, 60 leaves, max depth = 10, in 0.103s
[745/1000] 1 tree, 60 leaves, max depth = 10, in 0.085s
[746/1000] 1 tree, 60 leaves, max depth = 10, in 0.098s
[747/1000] 1 tree, 60 leaves, max depth = 10, in 0.080s
[748/1000] 1 tree, 159 leaves, max depth = 10, in 0.149s
[749/1000] 1 tree, 344 leaves, max depth = 10

[878/1000] 1 tree, 60 leaves, max depth = 10, in 0.090s
[879/1000] 1 tree, 349 leaves, max depth = 10, in 0.366s
[880/1000] 1 tree, 60 leaves, max depth = 10, in 0.078s
[881/1000] 1 tree, 191 leaves, max depth = 10, in 0.196s
[882/1000] 1 tree, 231 leaves, max depth = 10, in 0.313s
[883/1000] 1 tree, 209 leaves, max depth = 10, in 0.211s
[884/1000] 1 tree, 184 leaves, max depth = 10, in 0.184s
[885/1000] 1 tree, 147 leaves, max depth = 10, in 0.150s
[886/1000] 1 tree, 60 leaves, max depth = 10, in 0.093s
[887/1000] 1 tree, 60 leaves, max depth = 10, in 0.105s
[888/1000] 1 tree, 60 leaves, max depth = 10, in 0.106s
[889/1000] 1 tree, 60 leaves, max depth = 10, in 0.081s
[890/1000] 1 tree, 60 leaves, max depth = 10, in 0.095s
[891/1000] 1 tree, 246 leaves, max depth = 10, in 0.216s
[892/1000] 1 tree, 60 leaves, max depth = 10, in 0.100s
[893/1000] 1 tree, 60 leaves, max depth = 10, in 0.093s
[894/1000] 1 tree, 60 leaves, max depth = 10, in 0.102s
[895/1000] 1 tree, 60 leaves, max depth =

## Predicciones

In [18]:
ft.predecir(model, X_train, y_train, X_test, y_test)

Entrenamiento: 95.7976%
Testeo: 84.5923%.
Mean abs error: 480712.0030.


In [ ]:
plt.rcParams["figure.figsize"] = [15, 15]

importancia = model.feature_importances_

d = {'x':X_train.columns,'y':importancia}

df_importancia = pd.DataFrame(d)

In [ ]:
importancia = df_importancia.sort_values(by="y",ascending=False).head(40)
ax = plt.barh(importancia.x,importancia.y)


## Submission

In [ ]:
df_test = pd.read_csv('../../data/test.csv')


In [ ]:
df = df_test.copy()
df = ft.init_test(df)
#df = pd.merge(df,mx,on="provincia",how="left")

In [ ]:
df["ambientes"] = df["banos"] + df["habitaciones"]
df["ambientesygarage"] = df["banos"] + df["habitaciones"]+df["garages"]

In [ ]:
df[ ft.getTarget1Cols() ] = df[ft.getAllCols()+["antiguedad","banos","habitaciones"]]
df[ ft.getTarget2Cols() ] = df[["tipodepropiedad","idzona"]]
df[ ft.getTarget3Cols() ] = df[ft.getAllCols()]
df[ ft.getTarget4Cols() ] = df[ft.getAllCols()]
df[ ft.getTarget5Cols() ] = df[ft.getAllCols()]
df[ ft.getTarget6Cols() ] = df[ft.getAllCols()]

In [ ]:
df = ft.preprocess(df,  OHE, BE, 'test' , TE1, TE2, TE3, TE4, SumE, TE5, TE6)

In [ ]:
#df, X_train = df.align(X_train,axis=1)


In [ ]:
submit_prediction = model.predict( df )

In [ ]:
#cols_diff = []
#t = X_train_submit.columns.values
#fs = df.columns.values
#i = 0
#for feat in t:
#    if feat != fs[i]:
#        cols_diff.append(feat)
#    i=+1
#display(cols_diff)        

In [ ]:
#pd.concat([df_test[['id']], pd.Series(submit_prediction)], axis=1).rename(columns={0:"target"}).to_csv("../data/submission13.csv", index=False)

In [ ]:
from submission_helper import submission_output

output = submission_output(df_test, submit_prediction)

nombre_submit = 'submission39.csv'

output.to_csv(nombre_submit, index=False)